In [2]:
# Example use of the Sonic module Sputnik.jl

See https://julialang.org, https://juliadb.org and https://juliastats.org for reference.

In [ ]:
import Pkg                      # package manager
Pkg.activate("..")              # Project.toml
Pkg.instantiate()               # https://julialang.github.io/Pkg.jl/v1.0/index.html

In [ ]:
import Dates, JuliaDB, Random, StatsPlots

In [ ]:
using StatsPlots
ENV["GKSwstype"] = "100"
gr(fmt = :png)

In [ ]:
include("../src/Linckii.jl")
include("../src/LinckiiSecrets.jl")
include("../src/Sputnik.jl")

In [ ]:
access = Linckii.get_access(LinckiiSecrets.keab)

Load site,

In [ ]:
@time site = Linckii.loadsite(access, :nodes, :devices, :sensors)
()

In [ ]:
device_name = Linckii.get_cref(site.devices, :device_id, :device_name)
device_id   = Linckii.get_cref(site.devices, :device_name, :device_id)
()

In [ ]:
sensor_name = Linckii.get_cref(site.sensors, :sensor_id, :sensor_name)
sensor_id   = Linckii.get_cref(site.sensors, :sensor_name, :sensor_id)
()

Select data,

In [ ]:
selected_sensor_names = Dict(
    device_id["Kelp-Basic"] => [
        :meter_effect,
        :meter_heatenergy,
        :meter_primreturntemp,
        :meter_primsupplytemp,
        :meter_volume,
        :meter_volumeflow,
        :outdoortemp,
        :outdoortemp_offset,
    ],
)

In [ ]:
selected_nodes = JuliaDB.filter(
    r -> (
        r.device_id in keys(selected_sensor_names) &&
        Set(sensor_name[r_sensor_id] for r_sensor_id in r.sensor_ids) >=
        Set(selected_sensor_names[r.device_id])
    ),
    site.nodes,
)

Load data,

In [ ]:
rs = JuliaDB.rows(selected_nodes)
if true
    rs = [(node_id = 93, device_id = 4)] # [rs[Random.rand(1 : end)]]
end
data = Dict()
@time for r in rs
    filepath = "$(Linckii.datapath(access, r.node_id)).db"
    try
        data[r.node_id] = JuliaDB.load(filepath)
        println("$(filepath): Done")
    catch e
        println("$(filepath): $(e)")
    end
end

In [ ]:
# list different things to compute and plot across the selected substations